In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(2,6).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(6,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

In [3]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [4]:
dim1 = 4
dim2 = 6
dim3 = 3
bs = dim3

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.5

a = np.zeros((dim1,dim2))
b = np.zeros((dim2,dim3)).astype(np.float32)

a.shape, b.shape

((4, 6), (6, 3))

In [5]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, sparsity).astype(np.float32)
b = fill_sparse(b, sparsity).astype(np.float32)

In [6]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [7]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [8]:
x2_init.T

array([-0.03130801,  0.01457097, -0.5051717 ], dtype=float32)

In [9]:
mult = a.dot(b)
mult.shape

(4, 3)

In [10]:
mult.shape

(4, 3)

In [11]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = min(int(np.sqrt(np.max(all_nnzs))+1)**2, mat.shape[1])
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        #print(all_idxs[row])
    all_rows = np.array(all_rows)
    all_idxs = np.array(all_idxs)
    all_nnzs = np.array(all_nnzs)
    
    while (not all_rows[:,-1].any()):
        all_rows = all_rows[:,:-1]
        all_idxs = all_idxs[:,:-1]
        ellwidth -= 1
        
    
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [12]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [13]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

(array([0.48745027, 0.87602866, 0.57356703, 0.16462222, 0.41704008,
        0.6265748 , 0.5932638 , 0.17645018, 0.6961791 , 0.7643324 ,
        0.93176633, 0.62803787], dtype=float32),
 array([0, 2, 5, 1, 3, 4, 1, 2, 5, 3, 4, 5], dtype=uint32),
 array([3, 3, 3, 3], dtype=uint32),
 3)

In [14]:
adatat, acolst, annzt, ellwat = to_data(a.T)
adatat, acolst, annzt, ellwat

(array([0.48745027, 0.        , 0.        , 0.16462222, 0.5932638 ,
        0.        , 0.87602866, 0.17645018, 0.        , 0.41704008,
        0.7643324 , 0.        , 0.6265748 , 0.93176633, 0.        ,
        0.57356703, 0.6961791 , 0.62803787], dtype=float32),
 array([0, 0, 0, 1, 2, 0, 0, 2, 0, 1, 3, 0, 1, 3, 0, 0, 2, 3], dtype=uint32),
 array([1, 2, 2, 2, 2, 3], dtype=uint32),
 3)

In [15]:
bdata, bcols, bnnz, ellwb = to_data(b)
bdata, bcols, bnnz, ellwb

(array([0.4093198 , 0.27073938, 0.82387215, 0.26969674, 0.2974127 ,
        0.7703394 ], dtype=float32),
 array([2, 1, 1, 2, 2, 2], dtype=uint32),
 array([1, 1, 1, 1, 1, 1], dtype=uint32),
 1)

In [16]:
bdatat, bcolst, bnnzt, ellwbt = to_data(b.T)
adatat, bcolst, bnnzt, ellwbt

(array([0.48745027, 0.        , 0.        , 0.16462222, 0.5932638 ,
        0.        , 0.87602866, 0.17645018, 0.        , 0.41704008,
        0.7643324 , 0.        , 0.6265748 , 0.93176633, 0.        ,
        0.57356703, 0.6961791 , 0.62803787], dtype=float32),
 array([0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 4, 5], dtype=uint32),
 array([0, 2, 4], dtype=uint32),
 4)

In [17]:
adense = to_dense(adata, acols, annz, ellwa, a.shape)

In [18]:
adenset = to_dense(adatat, acolst, annzt, ellwat, a.T.shape)

In [19]:
bdense = to_dense(bdata, bcols, bnnz, ellwb, b.shape)

In [20]:
bdenset = to_dense(bdatat, bcolst, bnnzt, ellwbt, b.T.shape)

In [21]:
adense

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.62657481,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.69617909],
       [0.        , 0.        , 0.        , 0.76433241, 0.93176633,
        0.62803787]])

In [22]:
adenset.T == adense

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [23]:
bdenset.T == bdense

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [24]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [25]:
a == adense

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [26]:
a.shape

(4, 6)

In [27]:
adata.shape, acols.shape, annz.shape, ellwa

((12,), (12,), (4,), 3)

In [28]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [29]:
adata, acols, annz, b

(array([0.48745027, 0.87602866, 0.57356703, 0.16462222, 0.41704008,
        0.6265748 , 0.5932638 , 0.17645018, 0.6961791 , 0.7643324 ,
        0.93176633, 0.62803787], dtype=float32),
 array([0, 2, 5, 1, 3, 4, 1, 2, 5, 3, 4, 5], dtype=uint32),
 array([3, 3, 3, 3], dtype=uint32),
 array([[0.        , 0.        , 0.4093198 ],
        [0.        , 0.27073938, 0.        ],
        [0.        , 0.82387215, 0.        ],
        [0.        , 0.        , 0.26969674],
        [0.        , 0.        , 0.2974127 ],
        [0.        , 0.        , 0.7703394 ]], dtype=float32))

## MatMul (Sparse-Dense)

adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      float sum = 0;
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[col*ncols+gid2];
          //if (gid==0 && gid2==2)
          //  printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [30]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          uint xidx = col*ncols+gid2;
          float xval  = vector_x[xidx];
          if (gid==0 && gid2==1)
            printf("aval, xval: %.2f,%.2f: (%i,%i) - %i \\n", aval, xval, col, index, xidx);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [31]:
a.shape, b.shape

((4, 6), (6, 3))

In [32]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [33]:
rows = a.shape[0]

In [34]:
mult = mult.astype(np.float32)

In [35]:
outshape = (a.shape[0], b.shape[1])
outshape

(4, 3)

In [36]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul2  # Use this Kernel object for repeated calls
knl(queue, [outshape[0]], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), np.uint32(outshape[1]), b_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.49,0.00: (0,0) - 1 
aval, xval: 0.88,0.82: (2,1) - 7 
aval, xval: 0.57,0.00: (5,2) - 16 


In [37]:
(res_np-mult).sum()

0.0

In [38]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [39]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [40]:
res_buf

In [41]:
res_np

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [42]:
mult

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [43]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [44]:
res_np.shape

(4, 3)

In [45]:
mult.shape

(4, 3)

## MatMul (dense * sparse)

In [46]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        uint nnz = rowNnz[gid2];
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid2 * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid*mwidth+col];
          sum  += aval * xval;
          if (gid==0 && gid2==0)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [47]:
a.shape, b.shape

((4, 6), (6, 3))

In [48]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [49]:
rows = a.shape[0]

In [50]:
mult = mult.astype(np.float32)

In [51]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

array([4, 3])

In [52]:
b.T

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.27073938, 0.82387215, 0.        , 0.        ,
        0.        ],
       [0.4093198 , 0.        , 0.        , 0.26969674, 0.2974127 ,
        0.7703394 ]], dtype=float32)

In [53]:
a.T

array([[0.48745027, 0.        , 0.        , 0.        ],
       [0.        , 0.16462222, 0.5932638 , 0.        ],
       [0.87602866, 0.        , 0.17645018, 0.        ],
       [0.        , 0.41704008, 0.        , 0.7643324 ],
       [0.        , 0.6265748 , 0.        , 0.93176633],
       [0.57356703, 0.        , 0.6961791 , 0.62803787]], dtype=float32)

In [54]:
outshape.T

array([4, 3])

In [55]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [outshape.T[0]], None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), np.uint32(outshape.T[1]), a_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

In [56]:
(res_np-mult).sum()

0.0

In [57]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [58]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [59]:
res_buf

In [60]:
res_np

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [61]:
mult

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [62]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [63]:
res_np-mult

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [64]:
res_np.shape

(4, 3)

In [65]:
mult.shape

(4, 3)

# Matmult Dense Dense

In [66]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i; 
        float xval = x[xidx];
        uint yidx = osize*i+gidy;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [67]:
a.shape, b.shape

((4, 6), (6, 3))

In [68]:
rows = a.shape[0]

In [69]:
mult = mult.astype(np.float32)

In [70]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.49 x 0.00 - 0.00  -- 0/0
mult: 0.00 x 0.00 - 0.00  -- 1/3
mult: 0.88 x 0.00 - 0.00  -- 2/6
mult: 0.00 x 0.00 - 0.00  -- 3/9
mult: 0.00 x 0.00 - 0.00  -- 4/12
mult: 0.57 x 0.00 - 0.00  -- 5/15

In [71]:
(res_np-mult).sum()

0.0

In [72]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [73]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [74]:
res_np

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [75]:
a.dot(b)

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [76]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [77]:
res_np.shape

(4, 3)

In [78]:
mult.shape

(4, 3)

# Matmult Dense Transposed

In [79]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [80]:
c=np.zeros(b.T.shape)
bt = b.T
for row in range(bt.shape[0]):
    for col in range(bt.shape[1]):
        c[row][col] = bt[row][col]

In [81]:
bt

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.27073938, 0.82387215, 0.        , 0.        ,
        0.        ],
       [0.4093198 , 0.        , 0.        , 0.26969674, 0.2974127 ,
        0.7703394 ]], dtype=float32)

In [82]:
c

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.27073938, 0.82387215, 0.        , 0.        ,
        0.        ],
       [0.40931979, 0.        , 0.        , 0.26969674, 0.29741269,
        0.77033943]])

In [83]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i;
        float xval = x[xidx];
        uint yidx = msize*gidy+i;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [84]:
a.shape, b.T.shape

((4, 6), (3, 6))

In [85]:
rows = a.shape[0]

In [86]:
mult = mult.astype(np.float32)

In [87]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.49 x 0.00 - 0.00  -- 0/0
mult: 0.00 x 0.00 - 0.00  -- 1/1
mult: 0.88 x 0.00 - 0.00  -- 2/2
mult: 0.00 x 0.00 - 0.00  -- 3/3
mult: 0.00 x 0.00 - 0.00  -- 4/4
mult: 0.57 x 0.00 - 0.00  -- 5/5

In [88]:
(res_np-mult).sum()

0.0

In [89]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [90]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [91]:
res_np

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [92]:
a.dot(b)

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [93]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [94]:
res_np.shape

(4, 3)

In [95]:
mult.shape

(4, 3)

# Matmult Transposed Dense

In [96]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [97]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [98]:
at

array([[0.48745027, 0.        , 0.        , 0.        ],
       [0.        , 0.16462222, 0.5932638 , 0.        ],
       [0.87602866, 0.        , 0.17645018, 0.        ],
       [0.        , 0.41704008, 0.        , 0.7643324 ],
       [0.        , 0.6265748 , 0.        , 0.93176633],
       [0.57356703, 0.        , 0.6961791 , 0.62803787]], dtype=float32)

In [99]:
c

array([[0.48745027, 0.        , 0.        , 0.        ],
       [0.        , 0.16462222, 0.5932638 , 0.        ],
       [0.87602866, 0.        , 0.17645018, 0.        ],
       [0.        , 0.41704008, 0.        , 0.76433241],
       [0.        , 0.62657481, 0.        , 0.93176633],
       [0.57356703, 0.        , 0.69617909, 0.62803787]])

In [100]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint isize
                          ) { // LOCAL SHARED BUFFER
      uint osize = get_global_size(0);
      int gidy = get_global_id(0); // row
      
      for (uint gidx = 0; gidx < isize; gidx++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [101]:
a.shape, b.T.shape

((4, 6), (3, 6))

In [102]:
rows = a.shape[0]

In [103]:
mult = mult.astype(np.float32)

In [104]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [b.shape[1]], None, a_buf, b_buf, res_buf, np.uint32(a.shape[1]), np.uint32(rows))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.49 x 0.00 - 0.00  -- 0/0
mult: 0.00 x 0.00 - 0.00  -- 4/3
mult: 0.88 x 0.00 - 0.00  -- 8/6
mult: 0.00 x 0.00 - 0.00  -- 12/9
mult: 0.00 x 0.00 - 0.00  -- 16/12
mult: 0.57 x 0.00 - 0.00  -- 20/15

In [105]:
(res_np-mult).sum()

0.0

In [106]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [107]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [108]:
res_np

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [109]:
a.dot(b)

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [110]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [111]:
res_np.shape

(4, 3)

In [112]:
mult.shape

(4, 3)

# Matmult Dense Transposed2

In [113]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      // osize = get_global_size(1);
      int gidx = get_global_id(0); // col
      // int gidy = get_global_id(1); // row

      for (uint gidy = 0; gidy < osize; gidy++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          ret += x[gidx*msize+i]*y[i*osize+gidy];
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f", x[gidx*msize+i],y[i*msize+gidy], ret);
        }

        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [114]:
a.shape, b.shape

((4, 6), (6, 3))

In [115]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [116]:
rows = a.shape[0]

In [117]:
mult = mult.astype(np.float32)

In [118]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]), np.uint32(b.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.49 x 0.00 - 0.00
mult: 0.00 x 0.00 - 0.00
mult: 0.88 x 0.00 - 0.00
mult: 0.00 x 0.00 - 0.00
mult: 0.00 x 0.00 - 0.00
mult: 0.57 x 0.00 - 0.00

In [119]:
(res_np-mult).sum()

0.0

In [120]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [121]:
b.T

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.27073938, 0.82387215, 0.        , 0.        ,
        0.        ],
       [0.4093198 , 0.        , 0.        , 0.26969674, 0.2974127 ,
        0.7703394 ]], dtype=float32)

In [122]:
a[0]

array([0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
       0.57356703], dtype=float32)

In [123]:
b.T[0]

array([0., 0., 0., 0., 0., 0.], dtype=float32)

In [124]:
res_buf

In [125]:
res_np

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [126]:
a.dot(b)

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [127]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [128]:
res_np.shape

(4, 3)

In [129]:
mult.shape

(4, 3)

## Weight update kernel

In [130]:
bs = 4

In [131]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [132]:
dim = 8
topk = 2

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

((4, 8), (4, 8), 2)

x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        barrier(CLK_GLOBAL_MEM_FENCE);
        posx += (larger)?1:0;
        posy += (larger2)?1:0;
        barrier(CLK_GLOBAL_MEM_FENCE);
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          //printf("\\nJ:%i  gid:%i", j, gid);
          //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
          barrier(CLK_GLOBAL_MEM_FENCE);
          
        }
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
    }""").build()

In [133]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#x_cp_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
#x_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#y_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             uint bs,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      //uint bs = get_global_size(1);
      //uint gid2 = get_global_id(1);

      for (uint gid2=0; gid2<bs; gid2++){
        uint idx = n*gid2+gid;

        float valx = x[idx];
        float valy = y[idx];
        uint posx = 0;
        uint posy = 0;
        for (uint i = 0; i < n; i++) {
          uint idx2 = n*gid2+i;
          float tempval = x[idx2];
          float tempval2 = y[idx2];
          bool larger = tempval > valx;
          bool larger2 = tempval2 > valy;

          posx += (larger)?1:0;
          posy += (larger2)?1:0;
        }
        //printf("posx:%i", posx);
        if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
        }
        if (posy < topk) {
          youtidx[posy+topk*gid2] = gid;
        }
      }
      for (uint gid2=0; gid2<bs; gid2++){
        if (gid < topk) {
          for (uint j=0; j<topk; j++) {
            float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
            //printf("\\nJ:%i  gid:%i", j, gid);
            //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
            //barrier(CLK_GLOBAL_MEM_FENCE);
            xout[gid2*topk*topk+j*topk+gid] = res;
          }
        }
      }
    }""").build()

In [134]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
evt = knl(queue, [dim], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), np.uint32(bs), x_idx_buf, y_idx_buf)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

knl(queue, [dim], None, y_buf, x_buf, x_cp_buft, np.uint32(topk), np.uint32(bs), x_idx_buft, y_idx_buft)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [135]:
x

array([[0.9827091 , 0.9194645 , 0.4981366 , 0.66838795, 0.5474538 ,
        0.40575972, 0.9958302 , 0.20345955],
       [0.26779327, 0.6635567 , 0.3809771 , 0.77226824, 0.9726815 ,
        0.9388753 , 0.5080804 , 0.71193796],
       [0.40523973, 0.12542877, 0.7244864 , 0.4206373 , 0.3261672 ,
        0.4475811 , 0.6354743 , 0.61452025],
       [0.27658024, 0.39447892, 0.6531694 , 0.53033537, 0.18122154,
        0.5195999 , 0.35755244, 0.96476865]], dtype=float32)

In [136]:
y

array([[0.99472564, 0.9190837 , 0.54388493, 0.02103365, 0.61888707,
        0.48236775, 0.9965866 , 0.38499957],
       [0.65865105, 0.83802146, 0.6091217 , 0.2992236 , 0.86233723,
        0.10028819, 0.2559903 , 0.66472405],
       [0.67872417, 0.35487846, 0.8405793 , 0.43321013, 0.11889594,
        0.5728558 , 0.6004124 , 0.37214255],
       [0.39329135, 0.6985067 , 0.11035349, 0.407237  , 0.2939341 ,
        0.06044364, 0.6040797 , 0.56647724]], dtype=float32)

In [137]:
x.shape, y.shape

((4, 8), (4, 8))

In [138]:
resx

array([0.99243104, 0.97935474, 0.9905778 , 0.97752595, 0.8387795 ,
       0.8096271 , 0.81512797, 0.78679764, 0.6089883 , 0.5341665 ,
       0.49172643, 0.4313118 , 0.6738974 , 0.45624322, 0.58279717,
       0.3945664 ], dtype=float32)

In [139]:
resx.reshape(bs,topk,topk)

array([[[0.99243104, 0.97935474],
        [0.9905778 , 0.97752595]],

       [[0.8387795 , 0.8096271 ],
        [0.81512797, 0.78679764]],

       [[0.6089883 , 0.5341665 ],
        [0.49172643, 0.4313118 ]],

       [[0.6738974 , 0.45624322],
        [0.58279717, 0.3945664 ]]], dtype=float32)

In [140]:
resxidx

array([6, 0, 4, 5, 2, 6, 7, 2], dtype=uint32)

In [141]:
resyidx

array([6, 0, 4, 1, 2, 0, 1, 6], dtype=uint32)

In [142]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [143]:
xy0[3][7]

0.51334524

## Weight update kernel new

In [144]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [145]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [146]:
at

array([[0.48745027, 0.        , 0.        , 0.        ],
       [0.        , 0.16462222, 0.5932638 , 0.        ],
       [0.87602866, 0.        , 0.17645018, 0.        ],
       [0.        , 0.41704008, 0.        , 0.7643324 ],
       [0.        , 0.6265748 , 0.        , 0.93176633],
       [0.57356703, 0.        , 0.6961791 , 0.62803787]], dtype=float32)

In [147]:
c

array([[0.48745027, 0.        , 0.        , 0.        ],
       [0.        , 0.16462222, 0.5932638 , 0.        ],
       [0.87602866, 0.        , 0.17645018, 0.        ],
       [0.        , 0.41704008, 0.        , 0.76433241],
       [0.        , 0.62657481, 0.        , 0.93176633],
       [0.57356703, 0.        , 0.69617909, 0.62803787]])

a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             uint topk,
                             uint msize,
                             __global  float* xout,    // INPUT
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);



      uint idx = n*gid2+gid;
      float valx = x[idx];
      uint posx = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        bool larger = tempval > valx;
        posx += (larger)?1:0;
      }
      
      uint idxy = n*gid2+gid;
      float valy = y[idx];
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval2 = y[idx2];
        bool larger2 = tempval2 > valy;
        posy += (larger2)?1:0;
      }
      
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = idx;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = idxy;
      }
      return;
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*msize] * y[youtidx[j+topk*gid2]+gid2*msize];
          printf("\\nJ:%i  gid:(%i,%i)", j, gid, gid2);
          printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          //barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
        }
      }
    }""").build()

In [148]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topk,
                              __global  float* xout,
                              __global  uint* xoutidx,   
                              __global  uint* youtidx    
                              ) { 
      uint gid = get_global_id(0);
      
      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }
        
        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = xsum[i];
          bool larger = tempval > valx;
          posx += (larger)?1:0;
        }
        if (posx < topk) {
          xoutidx[posx] = gid;
        }
      }
      
      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          if (gid == 0) {
            printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          }
          ysum[gid] += val;
        }
        
        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = ysum[i];
          bool larger = tempval > valy;
          posy += (larger)?1:0;
        }
      
        if (posy < topk) {
          youtidx[posy] = gid;
        }
      }
      
      if (gid < topk) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[posx] = valx;
        
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[posy] = valy;
      }
      
      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < topk) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxy = youtidx[j];
          xout[j*topk+gid] = 0;
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            xout[j*topk+gid] += x[xidx2] * y[yidx2];
            //if (gid == 0 && j == 1)
            //  printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
        }
      }
    }""").build()

In [149]:
a.shape, b.shape

((4, 6), (6, 3))

In [150]:
rows = a.shape[0]
msize = a.shape[1]

In [151]:
cols = b.shape[1]

In [152]:
mult = a.dot(b)

In [153]:
mult = mult.astype(np.float32)

In [154]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate3  # Use this Kernel object for repeated calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), np.uint32(topk), x_cp_buf, x_idx_buf, y_idx_buf)

resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resx = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)


ADD VALx: 0.00 - 0
ADD VALx: 0.00 - 1
ADD VALx: 0.00 - 2
ADD VALx: 0.00 - 3
ADD VALx: 0.00 - 4
ADD VALx: 0.00 - 5

In [155]:
resx.reshape(topk,topk)

array([[0.7217356 , 0.        ],
       [0.64136434, 0.96705943]], dtype=float32)

In [156]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [157]:
b.T

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.27073938, 0.82387215, 0.        , 0.        ,
        0.        ],
       [0.4093198 , 0.        , 0.        , 0.26969674, 0.2974127 ,
        0.7703394 ]], dtype=float32)

In [158]:
resxsum

array([1.9370459, 1.2082372, 1.465893 , 2.3241367], dtype=float32)

In [159]:
resysum

array([0.       , 1.0946115, 1.7467687], dtype=float32)

In [160]:
a.sum(axis=1)

array([1.9370459, 1.2082372, 1.465893 , 2.3241367], dtype=float32)

In [161]:
b.sum(axis=0)

array([0.       , 1.0946115, 1.7467687], dtype=float32)

In [162]:
mult

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [163]:
resxidx

array([0, 3], dtype=uint32)

In [164]:
resyidx

array([1, 2], dtype=uint32)

In [165]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [166]:
xy0[0][0]

0.17638232

## Weight update kernel new2 (sparse ouput)

In [167]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [820]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
sdata_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topk*4)
sidxs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*topk*4)
snnzs_buf = cl.Buffer(ctx, mf.READ_WRITE, a.shape[0]*4)
sdatat_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topk*4)
sidxst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*topk*4)
snnzst_buf = cl.Buffer(ctx, mf.READ_WRITE, b.shape[1]*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate4(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topk,
                              __global  uint*  xoutidx,
                              __global  uint*  youtidx,
                              __global  float* matData,     // INPUT MATRIX DATA
                              __global  uint*  colIdx,
                              __global  uint*  rowNnz,
                              __global  float* matDatat,     // INPUT MATRIX DATA
                              __global  uint*  colIdxt,
                              __global  uint*  rowNnzt
                              ) { 
      uint gid = get_global_id(0);
      
      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = x[i*isize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, i*msize+gid);
          //}
          xsum[gid] += val;
        }
        
        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = xsum[i];
          bool larger = tempval > valx;
          posx += (larger)?1:0;
        }
        if (posx < topk) {
          xoutidx[posx] = gid;
        }
      }
      
      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<msize; i++) {
          float val = y[i*osize+gid];
          //if (gid == 0) {
          //  printf("\\nADD VALx: %.2f - %i", val, gid*osize+i);
          //}
          ysum[gid] += val;
        }
        
        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = ysum[i];
          bool larger = tempval > valy;
          posy += (larger)?1:0;
        }
      
        if (posy < topk) {
          youtidx[posy] = gid;
        }
      }
      
      if (gid < topk) {
        float valx = xoutidx[gid];
        uint posx = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = xoutidx[i];
          bool larger = tempval < valx;
          posx += (larger)?1:0;
        }
        xoutidx[posx] = valx;
        
        float valy = youtidx[gid];
        uint posy = 0;
        for (uint i = 0; i < topk; i++) {
          float tempval = youtidx[i];
          bool larger = tempval < valy;
          posy += (larger)?1:0;
        }
        youtidx[posy] = valy;
      }
      
      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < isize) {
        for (uint i=0; i<topk; i++) {
          matData[gid*topk+i] = 0;
          colIdx[gid*topk+i] = 0;
        }
        rowNnz[gid] = 0;  
      }
      if (gid < osize) {
        for (uint i=0; i<topk; i++) {
          matDatat[gid*topk+i] = 0;
          colIdxt[gid*topk+i] = 0;
        }
        rowNnzt[gid] = 0;  
      }
      
      
      if (gid < topk) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxy = youtidx[j];
          //printf("\\nIDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            uint colidx = idxy;
            matData[idxx*topk+j] += x[xidx2] * y[yidx2];
            colIdx[idxx*topk+j] = idxy;
            //if (gid == 0)
            //  printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnz[idxx] += 1;
        }
      }
      
      if (gid < topk) {
        uint idxy = youtidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxx = xoutidx[j];
          printf("\\nB-IDXX:%i  IDXY:%i", idxx, idxy);
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            uint colidx = idxy;
            float addval = x[xidx2] * y[yidx2];

            matDatat[idxy*topk+j] += addval;
            colIdxt[j*topk+idxx] = idxx;
            if (gid == 0)
              printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
          rowNnzt[idxy] += 1;
          printf("\\nAdd NNz:%i - %i", idxy, rowNnzt[idxy]);
        }
      }
    }""").build()

In [821]:
a.shape, b.shape

((4, 6), (6, 3))

In [822]:
rows = a.shape[0]
msize = a.shape[1]

In [823]:
cols = b.shape[1]

In [824]:
mult = a.dot(b)

In [825]:
mult = mult.astype(np.float32)

In [826]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate4  # Use this Kernel object for repeated calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), 
          np.uint32(topk), x_idx_buf, y_idx_buf, sdata_buf, sidxs_buf, snnzs_buf, sdatat_buf, sidxst_buf, snnzst_buf)

resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)
resxdat = np.zeros(a.shape[0]*topk).astype(np.float32)
resxcol = np.zeros(a.shape[0]*topk).astype(np.uint32)
resxnnz = np.zeros(a.shape[0]).astype(np.uint32)
resxdatt = np.zeros(b.shape[1]*topk).astype(np.float32)
resxcolt = np.zeros(b.shape[1]*topk).astype(np.uint32)
resxnnzt = np.zeros(b.shape[1]).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)
cl.enqueue_copy(queue, resxdat, sdata_buf)
cl.enqueue_copy(queue, resxcol, sidxs_buf)
cl.enqueue_copy(queue, resxnnz, snnzs_buf)
cl.enqueue_copy(queue, resxdatt, sdatat_buf)
cl.enqueue_copy(queue, resxcolt, sidxst_buf)
cl.enqueue_copy(queue, resxnnzt, snnzst_buf)


B-IDXX:0  IDXY:1
B-IDXX:0  IDXY:2
 ADD VAL:0.49,0.00 - (0,1) - (0,0,0)
 ADD VAL:0.00,0.27 - (0,1) - (0,0,1)
 ADD VAL:0.88,0.82 - (0,1) - (0,0,2)
 ADD VAL:0.00,0.00 - (0,1) - (0,0,3)
 ADD VAL:0.00,0.00 - (0,1) - (0,0,4)
 ADD VAL:0.57,0.00 - (0,1) - (0,0,5)
Add NNz:1 - 1
Add NNz:2 - 1
B-IDXX:3  IDXY:1
B-IDXX:3  IDXY:2
 ADD VAL:0.00,0.00 - (3,1) - (0,1,0)
 ADD VAL:0.00,0.27 - (3,1) - (0,1,1)
 ADD VAL:0.00,0.82 - (3,1) - (0,1,2)
 ADD VAL:0.76,0.00 - (3,1) - (0,1,3)
 ADD VAL:0.93,0.00 - (3,1) - (0,1,4)
 ADD VAL:0.63,0.00 - (3,1) - (0,1,5)
Add NNz:1 - 2
Add NNz:2 - 2

## results

In [827]:
mult

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [828]:
resxdat.reshape(a.shape[0],topk)

array([[0.7217356 , 0.64136434],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.96705943]], dtype=float32)

In [829]:
resxcol.reshape(a.shape[0],topk)

array([[1, 2],
       [0, 0],
       [0, 0],
       [1, 2]], dtype=uint32)

In [830]:
resxnnz.reshape(a.shape[0])

array([2, 0, 0, 2], dtype=uint32)

In [831]:
resxdatt.reshape(b.shape[1],topk)

array([[0.        , 0.        ],
       [0.7217356 , 0.        ],
       [0.64136434, 0.96705943]], dtype=float32)

In [818]:
resxcolt.reshape(b.shape[1],topk)

array([[0, 0],
       [0, 0],
       [0, 3]], dtype=uint32)

In [819]:
resxnnzt.reshape(b.shape[1])

array([0, 2, 2], dtype=uint32)

## comp

In [632]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [633]:
b

array([[0.        , 0.        , 0.4093198 ],
       [0.        , 0.27073938, 0.        ],
       [0.        , 0.82387215, 0.        ],
       [0.        , 0.        , 0.26969674],
       [0.        , 0.        , 0.2974127 ],
       [0.        , 0.        , 0.7703394 ]], dtype=float32)

In [634]:
resxsum

array([1.9370459, 1.2082372, 1.465893 , 2.3241367], dtype=float32)

In [635]:
a.sum(axis=1)

array([1.9370459, 1.2082372, 1.465893 , 2.3241367], dtype=float32)

In [636]:
resysum

array([0.       , 1.0946115, 1.7467687], dtype=float32)

In [637]:
b.sum(axis=0)

array([0.       , 1.0946115, 1.7467687], dtype=float32)

In [638]:
mult

array([[0.        , 0.7217356 , 0.64136434],
       [0.        , 0.04456972, 0.29882565],
       [0.        , 0.30599225, 0.5362942 ],
       [0.        , 0.        , 0.96705943]], dtype=float32)

In [639]:
resxidx

array([0, 3], dtype=uint32)

In [640]:
resyidx

array([1, 2], dtype=uint32)

In [641]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [642]:
xy0[0][0]

0.17638232

### update vals add dense

In [190]:
matadd = np.random.randn(*a.shape).astype(np.float32)
matadd

array([[-0.4501702 , -0.93507725, -1.5426744 ,  1.9229764 , -1.0766345 ,
         1.7863922 ],
       [ 0.05685159,  0.27631328,  0.5853075 , -0.16239385, -0.04222736,
         1.0666897 ],
       [ 0.20182703,  0.11041164,  0.39537126,  0.6592285 ,  0.24079415,
        -1.8186953 ],
       [ 0.457027  ,  0.1587936 , -0.8318175 , -0.76691616,  2.037848  ,
         0.31351304]], dtype=float32)

In [191]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [192]:
a_added = a + matadd

In [193]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   awidth,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*awidth;
      
      for (uint i=0; i<awidth; i++) {
        float addval = vector_x[baseidxd+i];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [194]:
a.shape, b.shape

((4, 6), (6, 3))

In [195]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [196]:
rows = a.shape[0]

In [197]:
mult = mult.astype(np.float32)

In [198]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa),np.uint32(a.shape[1]), add_buf)

In [199]:
matadd[0][0]

-0.4501702

In [200]:
data_res = np.empty_like(adata)
cols_res = np.empty_like(acols)
nnzs_res = np.empty_like(annz)
cl.enqueue_copy(queue, data_res, adata_buf)
cl.enqueue_copy(queue, cols_res, acols_buf)
cl.enqueue_copy(queue, nnzs_res, annzs_buf)

In [201]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, ellwa, a.shape)
adenseadd

array([[0.03728008, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.62657481,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.69617909],
       [0.        , 0.        , 0.        , 0.76433241, 0.93176633,
        0.62803787]])

In [202]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [203]:
matadd

array([[-0.4501702 , -0.93507725, -1.5426744 ,  1.9229764 , -1.0766345 ,
         1.7863922 ],
       [ 0.05685159,  0.27631328,  0.5853075 , -0.16239385, -0.04222736,
         1.0666897 ],
       [ 0.20182703,  0.11041164,  0.39537126,  0.6592285 ,  0.24079415,
        -1.8186953 ],
       [ 0.457027  ,  0.1587936 , -0.8318175 , -0.76691616,  2.037848  ,
         0.31351304]], dtype=float32)

In [204]:
a_added

array([[ 3.7280083e-02, -9.3507725e-01, -6.6664577e-01,  1.9229764e+00,
        -1.0766345e+00,  2.3599591e+00],
       [ 5.6851588e-02,  4.4093549e-01,  5.8530748e-01,  2.5464624e-01,
         5.8434743e-01,  1.0666897e+00],
       [ 2.0182703e-01,  7.0367545e-01,  5.7182145e-01,  6.5922850e-01,
         2.4079415e-01, -1.1225162e+00],
       [ 4.5702699e-01,  1.5879360e-01, -8.3181751e-01, -2.5837421e-03,
         2.9696143e+00,  9.4155091e-01]], dtype=float32)

In [205]:
adenseadd == a_added

array([[ True, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False]])

### update vals

In [206]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddenset(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   aheight,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint ncols = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      
      for (uint i=0; i<aheight; i++) {
        if (nnz > ellwidth)
            break;
        uint baseidxd = i*ncols+gid;
        float addval = vector_x[baseidxd];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
        }
      }
    }""").build()

In [207]:
a.shape, b.shape

((4, 6), (6, 3))

In [208]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [209]:
cols = a.shape[1]

In [210]:
mult = mult.astype(np.float32)

In [211]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddenset  # Use this Kernel object for repeated calls
knl(queue, [cols], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat),np.uint32(a.T.shape[1]), add_buf)


ADD VAL:-0.45 idx:0/0  col:0
ADD VAL:-1.54 idx:6/2  col:0
ADD VAL:1.79 idx:15/5  col:0
ADD VAL:0.28 idx:4/8  col:0
ADD VAL:-0.16 idx:10/10  col:0
ADD VAL:-0.04 idx:13/11  col:0
ADD VAL:0.11 idx:5/15  col:0
ADD VAL:0.40 idx:8/16  col:0

In [212]:
matadd[0][0]

-0.4501702

In [213]:
datat_res = np.empty_like(adatat)
colst_res = np.empty_like(acolst)
nnzst_res = np.empty_like(annzt)
cl.enqueue_copy(queue, datat_res, adatat_buf)
cl.enqueue_copy(queue, colst_res, acolst_buf)
cl.enqueue_copy(queue, nnzst_res, annzst_buf)

In [214]:
adenseaddt = to_dense(datat_res, colst_res, nnzst_res, ellwat, a.T.shape).T
adenseaddt

array([[ 0.03728008, -0.93507725, -0.66664577,  1.92297637, -1.07663453,
         2.35995913],
       [ 0.05685159,  0.44093549,  0.58530748,  0.25464624,  0.58434743,
         0.        ],
       [ 0.20182703,  0.70367545,  0.57182145,  0.        ,  0.        ,
         0.69617909],
       [ 0.        ,  0.        ,  0.        ,  0.76433241,  0.93176633,
         0.62803787]])

In [215]:
a

array([[0.48745027, 0.        , 0.87602866, 0.        , 0.        ,
        0.57356703],
       [0.        , 0.16462222, 0.        , 0.41704008, 0.6265748 ,
        0.        ],
       [0.        , 0.5932638 , 0.17645018, 0.        , 0.        ,
        0.6961791 ],
       [0.        , 0.        , 0.        , 0.7643324 , 0.93176633,
        0.62803787]], dtype=float32)

In [216]:
matadd

array([[-0.4501702 , -0.93507725, -1.5426744 ,  1.9229764 , -1.0766345 ,
         1.7863922 ],
       [ 0.05685159,  0.27631328,  0.5853075 , -0.16239385, -0.04222736,
         1.0666897 ],
       [ 0.20182703,  0.11041164,  0.39537126,  0.6592285 ,  0.24079415,
        -1.8186953 ],
       [ 0.457027  ,  0.1587936 , -0.8318175 , -0.76691616,  2.037848  ,
         0.31351304]], dtype=float32)

In [217]:
a_added

array([[ 3.7280083e-02, -9.3507725e-01, -6.6664577e-01,  1.9229764e+00,
        -1.0766345e+00,  2.3599591e+00],
       [ 5.6851588e-02,  4.4093549e-01,  5.8530748e-01,  2.5464624e-01,
         5.8434743e-01,  1.0666897e+00],
       [ 2.0182703e-01,  7.0367545e-01,  5.7182145e-01,  6.5922850e-01,
         2.4079415e-01, -1.1225162e+00],
       [ 4.5702699e-01,  1.5879360e-01, -8.3181751e-01, -2.5837421e-03,
         2.9696143e+00,  9.4155091e-01]], dtype=float32)

In [218]:
adenseaddt == a_added

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True, False, False, False],
       [False, False, False, False, False, False]])

### Make Random

In [219]:
rand = SparseTensor.uniform(2,4)
rand

<SparseTensor <GPUBuffer with shape (8,)> with grad None>

In [220]:
rand.to_numpy()

array([[0.00509653, 0.00227681, 0.00474796, 0.00318518],
       [0.00288702, 0.00722332, 0.00380243, 0.00371085]])

In [221]:
rand.data

<GPUBuffer with shape (8,)>

### update vals

In [222]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [223]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1)z;
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint col = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'GeForce GTX 1080 Ti' on 'NVIDIA CUDA' at 0x5645a7c33cb0>:

<kernel>:13:35: error: expected ';' at end of declaration
      uint gid2 = get_global_id(1)z;
                                  ^
                                  ;

(options: -I /home/fpaboim/.conda/envs/tinygrad/lib/python3.8/site-packages/pyopencl/cl)
(source saved as /tmp/tmpps54edlt.cl)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint topk = get_global_size(0);
      uint col = updateyidx[gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,1], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)

In [ ]:
resa.shape, resaidx.shape, resannz.shape, ellwa, a.T.shape

In [ ]:
adenseadd = to_dense(resa, resaidx, resannz, ellwa, a.shape)
adenseadd

In [ ]:
adenseadd - adense

In [ ]:
adenseadd == adense

In [ ]:
ellwa

In [ ]:
adata2 = adata.reshape(-1, ellwa)
adata2

In [ ]:
resa = resa.reshape(-1, ellwa)
resa

In [ ]:
resa - adata2

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

### update vals2

In [ ]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat), x_cp_buf, x_idx_buf, y_idx_buf)

resat = np.empty_like(adatat)
resaidxt = np.zeros(acolst.shape).astype(np.uint32)
resannzt = np.zeros(annzt.shape).astype(np.uint32)

cl.enqueue_copy(queue, resat, adatat_buf)
cl.enqueue_copy(queue, resaidxt, acolst_buf)
cl.enqueue_copy(queue, resannzt, annzst_buf)

In [ ]:
ellwa

In [ ]:
resat.shape, resaidxt.shape, resannzt.shape

In [ ]:
adenseaddt = to_dense(resat, resaidxt, resannzt, ellwat, a.T.shape)
adenseaddt

In [ ]:
adenseadd == adenseaddt.T

In [ ]:
adata2t = adatat.reshape(-1, ellwat)
adata2t

In [ ]:
resat = resat.reshape(-1, ellwat)
resat

In [ ]:
resat - adata2t

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
asdf

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""


In [ ]:
a2 = np.random.rand(4,4)

In [ ]:
a2

In [ ]:
a2.sum(axis=1)

In [ ]:
b2 = np.random.rand(4,4)

In [ ]:
b2

In [ ]:
b2.sum(axis=0)

In [ ]:
matmul = a2.dot(b2)
matmul